## Initial Pre-processing

This notebook demonstrates my pre-processing steps for water main break data. I download and import the most recently available data from the [regional database](https://open-kitchenergis.opendata.arcgis.com/datasets/KitchenerGIS::water-main-breaks/about) and get rid of the unnecessary columns so we can focus on the features that are going to essential for our model later on.

Let's dive in!

In [163]:
# Basic imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
%matplotlib inline

In [164]:
# Import our most recent data and check out a sample
data = "../data/raw/Nov_10_22_Water_Main_Breaks.csv"

break_data = pd.read_csv(data)
break_data.sample(10)

,X,Y,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,ROAD_CLOSED,SIDEWALK_CLOSED,HOUR_IMPACTED,UNITS_IMPACTED,...,CIVIC_NUMBER,STREET,ASSETID,ASSET_DEPTH,FROST_DEPTH,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
2697,-80.502843,43.423353,52801,145554,2022/03/03 10:39:37+00,MAIN,Partially Closed,Open,12-16 hours,0-50,...,387,WESTMOUNT RD E,42650,NaN,NaN,300.0,1968.0,DI,Y,786b5507-c0a7-4463-8e81-a4a5be8ea46f
2230,-80.486899,43.434152,10215,1136,2007/06/19 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,9,KEHL ST,134906,NaN,NaN,200.0,2017.0,PVCO,N,a4f18ded-5357-465e-9026-05b7408a918a
922,-80.516719,43.405269,8907,1998,2014/07/11 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,3,BERRY MOSS ST,62930,NaN,NaN,150.0,1999.0,PVC,Y,d9b8bfeb-6350-4bbb-ac6e-cd07942303ea
439,-80.460568,43.461327,8424,546,2004/02/25 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,203,ROSEMOUNT DR,33440,NaN,NaN,150.0,1959.0,CI,Y,4951c889-7bd1-46b1-b4c9-e6fb3e5ab5f7
321,-80.433821,43.434587,8306,643,1999/12/31 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,116,RUTHERFORD DR,33810,NaN,NaN,150.0,1964.0,CI,Y,397a9a68-ad08-4ede-accc-a9326711e9f4
817,-80.479139,43.432708,8802,128,1998/12/23 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,395,OTTAWA ST S,29680,NaN,NaN,NaN,NaN,NaN,N,4314f748-ee67-4cc1-bc86-b372b21a189d
1394,-80.459231,43.457360,9379,1287,2001/12/30 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,21,KENSINGTON AVE,21140,NaN,NaN,150.0,1954.0,CI,Y,b804d9b5-4c49-4e43-b2ab-637231dc2135
2352,-80.513657,43.423335,10337,2309,2017/12/16 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,29,UPLANDS DR,38860,NaN,NaN,150.0,1974.0,CI,Y,49adc186-1e45-4d1b-bf2c-c482cdcdf3c2
2181,-80.466996,43.450178,10166,225,2001/01/27 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,317,MELROSE AVE,76692,NaN,NaN,150.0,2010.0,PVC,N,af090f31-0a39-4ff1-b3b3-f025eeccf518
2259,-80.459771,43.438550,10244,2416,2018/01/01 00:00:00+00,MAIN,Open,Open,8-12 hours,NaN,...,801,WEBER ST E,41030,NaN,NaN,150.0,1952.0,CI,Y,198b1d9e-8e5e-4490-b0bc-185280db2b5c


In [165]:
break_data.shape

(2750, 52)

It seems like we have lots of columns that don't give us any useful information. Let's have a look and determine which columns we'd like to keep.

In [166]:
break_data.columns

Index(['X', 'Y', 'OBJECTID', 'WATBREAKINCIDENTID', 'INCIDENT_DATE',
       'BREAK_TYPE', 'ROAD_CLOSED', 'SIDEWALK_CLOSED', 'HOUR_IMPACTED',
       'UNITS_IMPACTED', 'CW_SERVICE_REQUEST', 'STATUS', 'STATUS_DATE',
       'WORKORDER', 'RETURN_TO_NORMAL', 'BREAK_NATURE', 'BREAK_APPARENT_CAUSE',
       'REPAIR_TYPE', 'NEW_SECTION_LENGTH', 'MAINTENANCE_DESC',
       'VALVES_CLOSED', 'VALVES_OPENED', 'HYDRANTS_CALLED_OUT',
       'HYDRANTS_CALLED_BACK_IN', 'POSITIVE_PRESSURE_MAINTANED',
       'AIR_GAP_MAINTANED', 'DISINFECTED', 'MECHANICAL_REMOVAL',
       'FLUSHING_EXCAVATION', 'HIGHER_VELOCITY_FLUSHING', 'ANODE_INSTALLED',
       'BREAK_CATEGORIZATION', 'BACTERIA_TESTING_DATE',
       'HEALTH_DEPT_NOTIFICATION', 'MOECC_SAC_NOTIFICATION',
       'SAC_REFERENCE_NO', 'LOCAL_MOE_OFFICE', 'BWA_DWA', 'BWA_DWA_DECLARED',
       'PROCEEDURES_FOLLOWED', 'RECORD_CHANGE_REQD', 'ROADSEGMENTID',
       'CIVIC_NUMBER', 'STREET', 'ASSETID', 'ASSET_DEPTH', 'FROST_DEPTH',
       'ASSET_SIZE', 'ASSET_YEAR_I

### Column description breakdown:

- Wat Break Incident ID
- Incident date
- Type of Asset Broken
- Does the road need to be closed?
- Does the sidewalk need to be closed?
- Estimated Hours for Repair
- Estimated Number of Units Impacted
- CW Service Request Number
- Current status of the break
- Status last updated date
- CW Workorder #
- Date operations was returned to normal service
- Nature of Break
- Apparent cause of break
- Repair Type
- New Section Length (m)
- Type of Planned Maintenance
- List Valves Closed
- List Hydrants Called Out
- List Hydrants Called Back In
- Positive Pressure Maintained?
- Air Gap Maintained thr Repair Process?
- Pipe and Press Parts Disinfected?
- Mechanical removal of contaminants?
- Flushing into the excavation?
- Higher velocity flushing after repairs?
- Anode Installed?
- Categorization of the Break
- Bacteria Testing Date Taken
- Health Dept Notification
- MOECC/SAC Notification
- Local MOE Office Notification
- BWA/DWA declared
- IF BWA / DWA are issued, Date/Time declared
- Where proceedures Followed?
- Was a record change required?
- Road Segment ID
- Closest Civic Number
- Street
- Related Asset ID
- Related Asset Depth (m)
- Depth of Frost (m)
- Asset Size (cm)
- Year Asset Installed
- Asset Material
- Asset Exists

We should check for null values in the features to see if it's worth imputing them or to just get rid of them altogether.

In [167]:
def print_null_values(df, column=None):
    if column is None:
        # print null values for all columns
        for col in df.columns:
            null_values = df[col].isnull().sum()
            print(f"{col} - ", null_values)
    else:
        # print null values for a single column
        null_values = df[column].isnull().sum()
        print(f"{col} - ", null_values)

In [168]:
# break_data.isna().sum()
print_null_values(break_data)

X -  0
Y -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
ROAD_CLOSED -  0
SIDEWALK_CLOSED -  0
HOUR_IMPACTED -  0
UNITS_IMPACTED -  2472
CW_SERVICE_REQUEST -  2730
STATUS -  0
STATUS_DATE -  167
WORKORDER -  1174
RETURN_TO_NORMAL -  2660
BREAK_NATURE -  117
BREAK_APPARENT_CAUSE -  168
REPAIR_TYPE -  2601
NEW_SECTION_LENGTH -  2733
MAINTENANCE_DESC -  2750
VALVES_CLOSED -  2749
VALVES_OPENED -  2750
HYDRANTS_CALLED_OUT -  2748
HYDRANTS_CALLED_BACK_IN -  2748
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
DISINFECTED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  133
BACTERIA_TESTING_DATE -  2743
HEALTH_DEPT_NOTIFICATION -  2750
MOECC_SAC_NOTIFICATION -  2747
SAC_REFERENCE_NO -  2750
LOCAL_MOE_OFFICE -  2747
BWA_DWA -  0
BWA_DWA_DECLARED -  2748
PROCEEDURES_FOLLOWED -  0
RECORD_CHANGE_REQD -  0
ROADSEGMENTID -  0
CIVIC_NUMBER -  97
STREET -  16
ASSETID -  0
ASSET_DEPTH

In [169]:
break_data = break_data.rename(columns={'X': 'LONGITUDE', 'Y': 'LATITUDE'})

In [170]:
break_data['INCIDENT_DATE'] = pd.to_datetime(break_data['INCIDENT_DATE'])

In [171]:
break_data.head()

,LONGITUDE,LATITUDE,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,ROAD_CLOSED,SIDEWALK_CLOSED,HOUR_IMPACTED,UNITS_IMPACTED,...,CIVIC_NUMBER,STREET,ASSETID,ASSET_DEPTH,FROST_DEPTH,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
0,-80.484005,43.462939,1,2252,2017-12-01 15:15:00+00:00,MAIN,Partially Closed,Open,12-16 hours,47,...,125,LANCASTER ST W,134292,1.6,0.3,450.0,1937.0,CI,Y,3521d297-1a2e-4e7b-a071-fc53ed87e965
1,-80.515075,43.422742,7874,1311,2001-03-26 00:00:00+00:00,SERVICE,Open,Open,8-12 hours,NaN,...,76,CLOVERDALE CRES,4101323,NaN,NaN,13.0,1965.0,XXX,Y,72445d62-16a8-43c1-9733-56b06015b077
2,-80.439811,43.445067,7875,1325,2006-09-06 00:00:00+00:00,SERVICE,Open,Open,8-12 hours,NaN,...,47,WREN CRES,4099987,NaN,NaN,25.0,1967.0,XXX,Y,3bdc8931-31c0-4090-a07a-a6847781dd97
3,-80.510859,43.426478,7876,1328,2006-09-11 00:00:00+00:00,SERVICE,Open,Open,8-12 hours,NaN,...,382,GREENBROOK DR,4642530,NaN,NaN,25.0,1964.0,PVC,Y,f75ad0b1-5b2a-4125-8ad5-2b9a037debd7
4,-80.457520,43.443201,7877,1308,2000-01-27 00:00:00+00:00,SERVICE,Open,Open,8-12 hours,NaN,...,224,MONTGOMERY RD,4100648,NaN,NaN,25.0,1967.0,XXX,Y,5a3c5d03-0899-4899-95e7-278bc5cbb682


After seeing how many null values some columns have and the type of information they display, it's safe to say that a lot can be dropped. Instead of going through and dropping all of them I'll write out what columns we'll want to keep.

In [172]:
break_data = break_data[['LONGITUDE', 'LATITUDE', 'OBJECTID', 'WATBREAKINCIDENTID', 'INCIDENT_DATE',
                         'BREAK_TYPE', 'HOUR_IMPACTED', 'STATUS', 'BREAK_NATURE',
                         'BREAK_APPARENT_CAUSE', 'POSITIVE_PRESSURE_MAINTANED', 'AIR_GAP_MAINTANED',
                         'MECHANICAL_REMOVAL', 'FLUSHING_EXCAVATION', 'HIGHER_VELOCITY_FLUSHING', 'ANODE_INSTALLED',
                         'BREAK_CATEGORIZATION', 'ROADSEGMENTID', 'STREET', 'ASSETID', 'ASSET_SIZE', 'ASSET_YEAR_INSTALLED',
                         'ASSET_MATERIAL', 'ASSET_EXISTS', 'GLOBALID']]

In [173]:
print(break_data.shape)
break_data.sample(5)

(2750, 25)


,LONGITUDE,LATITUDE,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,HOUR_IMPACTED,STATUS,BREAK_NATURE,BREAK_APPARENT_CAUSE,...,ANODE_INSTALLED,BREAK_CATEGORIZATION,ROADSEGMENTID,STREET,ASSETID,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
770,-80.447364,43.425269,8755,2298,2017-12-01 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,104864,FAIRLAWN RD,13800,150.0,1964.0,CI,Y,4cf1edab-bf88-4bba-a3d9-786d9643efe4
804,-80.490018,43.477562,8789,1256,2008-12-01 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,12586,LANG CRES,24390,150.0,1948.0,CI,N,1098da2d-de7d-49ed-a3a0-d7970f0151ad
845,-80.431917,43.432166,8830,934,2000-12-22 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,23060,JANSEN AVE,20580,150.0,1970.0,CI,Y,c3dfb7d8-9678-4dae-a637-1d9d2039ae63
2354,-80.436134,43.433234,10339,2311,2017-12-22 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,23056,KINZIE AVE,22500,150.0,1961.0,CI,Y,0bc06989-9eb7-4281-9a20-d5762eff457c
2079,-80.450134,43.424308,10064,1559,2010-12-27 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,CIRCUMFERENTIAL,COMBINATION,...,Y,CATEGORY 1,22946,BYRON AVE,92734,150.0,1954.0,CI,Y,2033ec20-8243-4db4-9821-fb5348cc0859


How many null values do we still need to take care of?

In [174]:
print_null_values(break_data)

LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  117
BREAK_APPARENT_CAUSE -  168
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  133
ROADSEGMENTID -  0
STREET -  16
ASSETID -  0
ASSET_SIZE -  161
ASSET_YEAR_INSTALLED -  165
ASSET_MATERIAL -  161
ASSET_EXISTS -  0
GLOBALID -  0


Let's investigate the types of values there are in the `BREAK_APPARENT_CAUSE` variable and `BREAK_NATURE` variable. There aren't a lot of missing values so I might be able to easily impute them.

In [175]:
break_data.BREAK_APPARENT_CAUSE.unique()

array(['AGE', 'OTHER', 'COMBINATION', 'CORROSION', 'SOILS', 'UNKNOWN',
       'PRESSURE', 'FAULTY INSTALL', nan], dtype=object)

In [176]:
break_data.BREAK_APPARENT_CAUSE.value_counts()

OTHER             2006
AGE                232
COMBINATION        146
CORROSION          112
UNKNOWN             34
PRESSURE            22
SOILS               21
FAULTY INSTALL       9
Name: BREAK_APPARENT_CAUSE, dtype: int64

I'll fill the nan values with 'UNKNOWN'

In [177]:
break_data.BREAK_APPARENT_CAUSE.fillna('UNKNOWN', inplace=True)

In [178]:
break_data.BREAK_NATURE.value_counts()

UNKNOWN                                      2005
CIRCUMFERENTIAL                               370
CORROSION                                      87
FITTING/JOINT                                  55
LONGITUDINAL                                   29
CIRCUMFERENTIAL AND FITTING/JOINT              26
CORROSION AND CIRCUMFERENTIAL                  18
OTHER                                          14
OTHER: WATER SERVICE                            9
CORROSION AND LONGITUDINAL                      9
CORROSION AND FITTING/JOINT                     6
FITTING/JOINT AND LONGITUDINAL                  4
CORROSION - ROBAR SADDLE CORRODED AT SEAM       1
Name: BREAK_NATURE, dtype: int64

In [179]:
break_data['BREAK_NATURE'] = break_data['BREAK_NATURE'].replace({'OTHER: WATER SERVICE': 'WATER SERVICE'})

In [180]:
break_data.BREAK_NATURE.value_counts()

UNKNOWN                                      2005
CIRCUMFERENTIAL                               370
CORROSION                                      87
FITTING/JOINT                                  55
LONGITUDINAL                                   29
CIRCUMFERENTIAL AND FITTING/JOINT              26
CORROSION AND CIRCUMFERENTIAL                  18
OTHER                                          14
WATER SERVICE                                   9
CORROSION AND LONGITUDINAL                      9
CORROSION AND FITTING/JOINT                     6
FITTING/JOINT AND LONGITUDINAL                  4
CORROSION - ROBAR SADDLE CORRODED AT SEAM       1
Name: BREAK_NATURE, dtype: int64

In [181]:
break_data.BREAK_NATURE.unique()

array(['CORROSION AND FITTING/JOINT', 'UNKNOWN', 'CORROSION',
       'CIRCUMFERENTIAL', 'CORROSION AND CIRCUMFERENTIAL',
       'FITTING/JOINT', 'CIRCUMFERENTIAL AND FITTING/JOINT',
       'LONGITUDINAL', 'WATER SERVICE', 'CORROSION AND LONGITUDINAL',
       'FITTING/JOINT AND LONGITUDINAL', 'OTHER', nan,
       'CORROSION - ROBAR SADDLE CORRODED AT SEAM'], dtype=object)

In [182]:
break_data.BREAK_NATURE.fillna('UNKNOWN', inplace=True)

In [183]:
break_data.BREAK_NATURE.value_counts()

UNKNOWN                                      2122
CIRCUMFERENTIAL                               370
CORROSION                                      87
FITTING/JOINT                                  55
LONGITUDINAL                                   29
CIRCUMFERENTIAL AND FITTING/JOINT              26
CORROSION AND CIRCUMFERENTIAL                  18
OTHER                                          14
WATER SERVICE                                   9
CORROSION AND LONGITUDINAL                      9
CORROSION AND FITTING/JOINT                     6
FITTING/JOINT AND LONGITUDINAL                  4
CORROSION - ROBAR SADDLE CORRODED AT SEAM       1
Name: BREAK_NATURE, dtype: int64

Most research papers that I've read so far really only include pipe attributes (length, width, diameter, etc.), last break incident, type of break, number of previous breaks, material of pipe, as well as other data not directly related to the pipe. This includes soil moisture, soil resistivity, soil corrosivity, average temperature, and other attributes like this.

Disregarding the data about soil, there are still a decent number of features here that don't seem like they would contribute much information to the model.

In [184]:
print_null_values(break_data)

LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  0
BREAK_APPARENT_CAUSE -  0
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  133
ROADSEGMENTID -  0
STREET -  16
ASSETID -  0
ASSET_SIZE -  161
ASSET_YEAR_INSTALLED -  165
ASSET_MATERIAL -  161
ASSET_EXISTS -  0
GLOBALID -  0


In [185]:
break_data.BREAK_CATEGORIZATION.value_counts()

CATEGORY 1    2603
CATEGORY 2      14
Name: BREAK_CATEGORIZATION, dtype: int64

In [186]:
def fill_null_values(df, column, value):
    # fill null values in the specified column with the specified value
    df[column].fillna(value, inplace=True)

In [187]:
# break_data.BREAK_CATEGORIZATION.fillna('UNKNOWN', inplace=True)
fill_null_values(break_data, 'BREAK_CATEGORIZATION', 'UNKNOWN')

In [188]:
break_data.BREAK_CATEGORIZATION.value_counts()

CATEGORY 1    2603
UNKNOWN        133
CATEGORY 2      14
Name: BREAK_CATEGORIZATION, dtype: int64

We looked at 'BREAK_NATURE' earlier in this notebook, but I'm just noticing now that the feature has both 'UNKNOWN' and 'OTHER' as values. We need to convert these to one or the other so everything stays consistent. Let's first see all of the unique values again, and then fix this issue. Since there are a significant amount of the 'UNKNOWN' value we'll most likely convert 'OTHER' to that.

In [189]:
break_data.BREAK_NATURE.unique()

array(['CORROSION AND FITTING/JOINT', 'UNKNOWN', 'CORROSION',
       'CIRCUMFERENTIAL', 'CORROSION AND CIRCUMFERENTIAL',
       'FITTING/JOINT', 'CIRCUMFERENTIAL AND FITTING/JOINT',
       'LONGITUDINAL', 'WATER SERVICE', 'CORROSION AND LONGITUDINAL',
       'FITTING/JOINT AND LONGITUDINAL', 'OTHER',
       'CORROSION - ROBAR SADDLE CORRODED AT SEAM'], dtype=object)

In [190]:
break_data['BREAK_NATURE'] = break_data['BREAK_NATURE'].replace({'OTHER': 'UNKNOWN'})

In [191]:
break_data.BREAK_NATURE.unique()

array(['CORROSION AND FITTING/JOINT', 'UNKNOWN', 'CORROSION',
       'CIRCUMFERENTIAL', 'CORROSION AND CIRCUMFERENTIAL',
       'FITTING/JOINT', 'CIRCUMFERENTIAL AND FITTING/JOINT',
       'LONGITUDINAL', 'WATER SERVICE', 'CORROSION AND LONGITUDINAL',
       'FITTING/JOINT AND LONGITUDINAL',
       'CORROSION - ROBAR SADDLE CORRODED AT SEAM'], dtype=object)

In [192]:
break_data.BREAK_NATURE.value_counts()

UNKNOWN                                      2136
CIRCUMFERENTIAL                               370
CORROSION                                      87
FITTING/JOINT                                  55
LONGITUDINAL                                   29
CIRCUMFERENTIAL AND FITTING/JOINT              26
CORROSION AND CIRCUMFERENTIAL                  18
WATER SERVICE                                   9
CORROSION AND LONGITUDINAL                      9
CORROSION AND FITTING/JOINT                     6
FITTING/JOINT AND LONGITUDINAL                  4
CORROSION - ROBAR SADDLE CORRODED AT SEAM       1
Name: BREAK_NATURE, dtype: int64

In [193]:
print_null_values(break_data)

LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  0
BREAK_APPARENT_CAUSE -  0
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  0
ROADSEGMENTID -  0
STREET -  16
ASSETID -  0
ASSET_SIZE -  161
ASSET_YEAR_INSTALLED -  165
ASSET_MATERIAL -  161
ASSET_EXISTS -  0
GLOBALID -  0


We can see there's some `STREET` names missing. It would be best to explore these missing attributes and see how we can possibly impute them. They still contain `LATITUDE` and `LONGITUDE` values so we can possibly identify them by those. Let's see what we can do...

In [194]:
no_street = break_data.loc[break_data.STREET.isna()]
no_street

,LONGITUDE,LATITUDE,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,HOUR_IMPACTED,STATUS,BREAK_NATURE,BREAK_APPARENT_CAUSE,...,ANODE_INSTALLED,BREAK_CATEGORIZATION,ROADSEGMENTID,STREET,ASSETID,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
155,-80.573004,43.443999,8140,1405,2009-10-22 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,6b0439bb-b9db-4144-b228-43b612d85586
293,-80.474160,43.471444,8278,976,2001-03-14 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,604217,NaN,32860,200.0,1974.0,DI,Y,b3b77f1f-124e-4653-bb23-32aa195500b9
493,-80.427697,43.390080,8478,1978,2014-01-10 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,50156,NaN,91992,200.0,1977.0,DI,Y,6b29d708-e81b-4808-877c-31c686c9fd8c
693,-80.573004,43.443999,8678,505,2011-12-15 15:38:27+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,7ce0a441-0385-45b6-b915-6c6d1a8c0876
1443,-80.573004,43.443999,9428,1330,2011-12-15 15:38:28+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,54d89f5a-f3c7-4a9f-9a64-80a6696ff96b
1607,-80.573004,43.443999,9592,1329,2011-12-15 15:38:28+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,96d2114a-1e34-439c-8409-a42c298fe957
1616,-80.573004,43.443999,9601,1403,2009-10-14 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,f79cd49f-967d-478a-97fc-b4ac000d5d24
1758,-80.457321,43.458436,9743,1335,2009-01-10 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,603891,NaN,22910,150.0,1955.0,CI,Y,58888261-25da-4173-b479-a0a6f5ae2fdc
2039,-80.573004,43.443999,10024,725,2011-12-15 15:38:27+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,73973bb7-ceba-45aa-b61f-f4900024a35d
2052,-80.573004,43.443999,10037,7,2011-12-15 15:38:26+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,COMBINATION,...,Y,CATEGORY 1,0,NaN,0,NaN,NaN,NaN,N,0ddf6145-505e-4b7b-bd1b-200fafd74b6c


Looking at the latitude and longitude of the missing street addresses, there are actually only 5 distinct missing addresses. We can plot them on a map and see where they are located.

In [195]:
import plotly
import plotly.express as px

In [196]:
from config import token
api_token = token
px.set_mapbox_access_token(token)
fig = px.scatter_mapbox(data_frame=no_street, lat='LATITUDE', lon='LONGITUDE')
fig.update_layout(mapbox_style="carto-positron", mapbox_accesstoken=token)
fig.show();

Plugging the latitude and longitudes that are missing into a reverse geocoding website, we can get the following addresses that we can plug into the data.

In [197]:
break_data.loc[(break_data['LONGITUDE'] == -80.573004) & (break_data['LATITUDE'] == 43.443999), 'STREET'] = 'ERB ST'
break_data.loc[(break_data['LONGITUDE'] == -80.427697) & (break_data['LATITUDE'] == 43.390080), 'STREET'] = 'OLD CARRIAGE DR'
break_data.loc[(break_data['LONGITUDE'] == -80.457321) & (break_data['LATITUDE'] == 43.458436), 'STREET'] = 'KRUG ST'
break_data.loc[(break_data['LONGITUDE'] == -80.294712) & (break_data['LATITUDE'] == 43.538106), 'STREET'] = 'SPEEDVALE AVE W'

In [198]:
break_data.STREET.fillna('UNKNOWN', inplace=True)

In [199]:
print_null_values(break_data)

LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  0
BREAK_APPARENT_CAUSE -  0
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  0
ROADSEGMENTID -  0
STREET -  0
ASSETID -  0
ASSET_SIZE -  161
ASSET_YEAR_INSTALLED -  165
ASSET_MATERIAL -  161
ASSET_EXISTS -  0
GLOBALID -  0


In [200]:
break_data.ASSET_SIZE.value_counts()

150.0     1808
300.0      319
200.0      289
100.0       58
450.0       51
600.0       17
25.0        13
250.0       12
50.0         7
13.0         5
1200.0       5
0.0          3
750.0        2
Name: ASSET_SIZE, dtype: int64

So there's 121 missing values for `ASSET_SIZE`. We could simply fill the null values with the mean value, the most frequent value, or just set them to 0. Another way that I feel might be the most appropriate approach is visualize on a map where these asset sizes occur, categorizing by street name, and then seeing if the null values occur on the same streets as some of those already known asset sizes, and then fill in the missing sizes with which neighbours their near.

In [201]:
# pd.set_option("display.max_rows", None)
break_data.loc[break_data.ASSET_SIZE.isna()]

,LONGITUDE,LATITUDE,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,HOUR_IMPACTED,STATUS,BREAK_NATURE,BREAK_APPARENT_CAUSE,...,ANODE_INSTALLED,BREAK_CATEGORIZATION,ROADSEGMENTID,STREET,ASSETID,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
8,-80.486628,43.436634,7881,2047,2015-03-12 00:00:00+00:00,SERVICE,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11399,MILL ST,4101475,NaN,NaN,NaN,N,536f67ed-4ec6-470e-b0d2-7e916e639d6a
22,-80.483756,43.435221,8007,67,2003-04-28 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11413,MILL ST,26730,NaN,NaN,NaN,N,2b7d9979-3508-4727-b20d-54a50b6a4e9c
27,-80.435665,43.431450,8012,1113,2007-03-07 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,12095,FLORENCE AVE,14690,NaN,NaN,NaN,N,d9a5c6eb-e57a-4e2d-84b0-0962f6faa35e
43,-80.479748,43.431306,8028,1384,2009-04-30 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11743,OTTAWA ST S,95460,NaN,NaN,NaN,N,e9f999c9-9b13-4951-aee7-baea12cd9d9e
83,-80.480002,43.431070,8068,266,1999-01-27 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11743,OTTAWA ST S,29660,NaN,NaN,NaN,N,1c8c083c-e54d-44db-8b6c-91973db0604c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,-80.489597,43.467957,33925,140294,2021-01-16 08:30:32+00:00,MAIN,4-8 hours,CANCELLED,UNKNOWN,UNKNOWN,...,Y,UNKNOWN,10427,BOEHMER ST,3680,NaN,NaN,NaN,Y,4ab4f0f3-960e-4a8d-882e-57dbec087fbe
2578,-80.489597,43.467885,33926,140296,2021-01-16 08:31:03+00:00,MAIN,4-8 hours,REPAIR COMPLETED,UNKNOWN,UNKNOWN,...,Y,UNKNOWN,10430,BOEHMER ST,3680,NaN,NaN,NaN,N,72a3e429-c580-4d7b-a8a5-ef6e81f20639
2595,-80.294712,43.538106,36482,140716,2021-02-17 09:53:29+00:00,MAIN,20-24 hours,CANCELLED,UNKNOWN,UNKNOWN,...,Y,UNKNOWN,0,UNKNOWN,0,NaN,NaN,NaN,Y,c3e03c5c-1489-4eb2-88e4-645ea92bdf8c
2596,-80.294712,43.538106,36483,140718,2021-02-17 10:03:55+00:00,MAIN,20-24 hours,CANCELLED,UNKNOWN,UNKNOWN,...,Y,UNKNOWN,0,UNKNOWN,0,NaN,NaN,NaN,Y,1ceaeaa1-570c-4a84-ab3a-8c0b1568d6af


Unfortunately 13 out of the previous 16 null values for street names occur in this set. But that's okay for those ones, it just means that we might only end up with 13 null values instead of the current 161. It's better to figure out how to fill in the majority of the values and worry about those few afterwards.

What I will try to do next is look at a sort of breakdown of the asset sizes, listed with their asset ID's and corresponding street names, and see if we can match up any non-null asset sizes from the same street as asset sizes with null values. There's a possibility of imputing the average asset size value from the same street for the null values. For now I am jusy hypothesizing this, I'm not entirely sure if it'll work but why not try right?

In [202]:
asset_size = break_data[['STREET', 'ASSETID', 'ASSET_SIZE']]
# cherry picking null asset sizes to see which streets we could look at
asset_size.loc[asset_size.ASSET_SIZE.isna()].sample(10)

,STREET,ASSETID,ASSET_SIZE
2418,SCHWEITZER ST,442,NaN
1421,BRIDGE ST E,444,NaN
1572,OTTAWA ST S,29680,NaN
2326,OTTAWA ST S,29680,NaN
996,BECKER ST,95100,NaN
2039,UNKNOWN,0,NaN
244,FLORENCE AVE,14680,NaN
2505,MILL ST,4101474,NaN
1253,GOLFVIEW PL,140414,NaN
817,OTTAWA ST S,29680,NaN


Next I'll see if there are any matching asset ID's for null and non-null asset sizes. If there are any matches, then it could be safe to say that I could impute the null values with the ID's matching asset size. If this doesn't prove to be true then I'll have to explore more options.

In [203]:
asset_size[asset_size['STREET'] == 'FLORENCE AVE']

,STREET,ASSETID,ASSET_SIZE
27,FLORENCE AVE,14690,NaN
147,FLORENCE AVE,14700,NaN
244,FLORENCE AVE,14680,NaN
288,FLORENCE AVE,14690,NaN
738,FLORENCE AVE,14680,NaN
1313,FLORENCE AVE,14680,NaN
1410,FLORENCE AVE,14700,NaN
1454,FLORENCE AVE,14690,NaN
1518,FLORENCE AVE,14700,NaN
1734,FLORENCE AVE,14690,NaN


Unfortunately there are no matching ID's for this street. Let's check a few more streets to be sure that this trend might not hold...

In [204]:
asset_size[asset_size['STREET'] == 'OTTAWA ST S']

,STREET,ASSETID,ASSET_SIZE
43,OTTAWA ST S,95460,NaN
83,OTTAWA ST S,29660,NaN
395,OTTAWA ST S,64330,NaN
490,OTTAWA ST S,95460,NaN
523,OTTAWA ST S,29630,150.0
596,OTTAWA ST S,29620,150.0
699,OTTAWA ST S,29620,150.0
748,OTTAWA ST S,95460,NaN
753,OTTAWA ST S,29660,NaN
805,OTTAWA ST S,29630,150.0


In [205]:
# display street names with no asset size
streets_with_na = asset_size.STREET[asset_size['ASSET_SIZE'].isna()].unique()
print(streets_with_na)

['MILL ST' 'FLORENCE AVE' 'OTTAWA ST S' 'WEBER ST E' 'OTTAWA ST N'
 'UNKNOWN' 'ST CLAIR AVE' 'BRIDGE ST E' 'CORAL CRES' 'BOEHMER ST'
 'MAUSSER AVE' 'VALEWOOD PL' 'WINDOM RD' 'REX DR' 'NORFOLK CRES' 'KEHL ST'
 'HEIMAN ST' 'HEBEL PL' 'GUERIN AVE' 'BECKER ST' 'STIRLING AVE S'
 'MAURICE ST' 'SOUTHILL DR' 'WALKER ST' 'EIGHTH AVE' 'FAIRMOUNT RD'
 'FERGUS AVE' 'HUBER ST' 'GOLFVIEW PL' 'PATTANDON AVE' 'HOFFMAN ST'
 'SYDNEY ST S' 'ANN ST' 'EDWIN ST' 'SCHWEITZER ST']


In [206]:
asset_size[asset_size['STREET'] == 'WEBER ST E']

,STREET,ASSETID,ASSET_SIZE
29,WEBER ST E,41030,150.0
101,WEBER ST E,79238,NaN
110,WEBER ST E,40960,NaN
263,WEBER ST E,40820,NaN
310,WEBER ST E,41030,150.0
499,WEBER ST E,40760,NaN
528,WEBER ST E,41030,150.0
565,WEBER ST E,40960,NaN
664,WEBER ST E,40960,NaN
849,WEBER ST E,40960,NaN


Well there we have it, each ID is unique to the asset size whether it's missing or not.

I will fill in the `NaN` values with the most frequent number (mode) for now.

In [207]:
asset_size.ASSET_SIZE.value_counts()

150.0     1808
300.0      319
200.0      289
100.0       58
450.0       51
600.0       17
25.0        13
250.0       12
50.0         7
13.0         5
1200.0       5
0.0          3
750.0        2
Name: ASSET_SIZE, dtype: int64

In [208]:
# fill asset size with the mode of the column
break_data['ASSET_SIZE'].fillna(break_data['ASSET_SIZE'].mode()[0], inplace=True)

In [209]:
break_data.ASSET_SIZE.value_counts()

150.0     1969
300.0      319
200.0      289
100.0       58
450.0       51
600.0       17
25.0        13
250.0       12
50.0         7
13.0         5
1200.0       5
0.0          3
750.0        2
Name: ASSET_SIZE, dtype: int64

In [210]:
print_null_values(break_data)

LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  0
BREAK_APPARENT_CAUSE -  0
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  0
ROADSEGMENTID -  0
STREET -  0
ASSETID -  0
ASSET_SIZE -  0
ASSET_YEAR_INSTALLED -  165
ASSET_MATERIAL -  161
ASSET_EXISTS -  0
GLOBALID -  0


In [211]:
break_data.ASSET_YEAR_INSTALLED.value_counts()

1966.0    159
1958.0    158
1967.0    156
1962.0    110
1953.0    104
         ... 
1904.0      1
1990.0      1
1936.0      1
1997.0      1
1920.0      1
Name: ASSET_YEAR_INSTALLED, Length: 94, dtype: int64

In [212]:
break_data[break_data['ASSET_YEAR_INSTALLED'].isna()].sample(10)

,LONGITUDE,LATITUDE,OBJECTID,WATBREAKINCIDENTID,INCIDENT_DATE,BREAK_TYPE,HOUR_IMPACTED,STATUS,BREAK_NATURE,BREAK_APPARENT_CAUSE,...,ANODE_INSTALLED,BREAK_CATEGORIZATION,ROADSEGMENTID,STREET,ASSETID,ASSET_SIZE,ASSET_YEAR_INSTALLED,ASSET_MATERIAL,ASSET_EXISTS,GLOBALID
512,-80.475181,43.479341,8497,1006,2006-12-19 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,13107,BRIDGE ST E,444,150.0,NaN,NaN,N,e82d2806-040a-40aa-a8fa-ead2987f758b
469,-80.475473,43.434025,8454,1196,2008-02-10 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11455,MILL ST,26700,150.0,NaN,NaN,N,bb133a77-a561-459f-8b86-9cdc6fe2c784
986,-80.436979,43.432027,8971,1174,2007-12-18 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11544,HUBER ST,19990,150.0,NaN,NaN,N,594ccac5-e1aa-4fa6-941d-b5868584798b
263,-80.485117,43.450855,8248,1540,2010-11-09 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,FITTING/JOINT,AGE,...,Y,CATEGORY 1,6759,WEBER ST E,40820,150.0,NaN,NaN,N,2d17eae0-78e2-4e45-9871-725d0ccb044c
2416,-80.466276,43.481911,10401,653,2003-02-26 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,12946,SCHWEITZER ST,441,150.0,NaN,NaN,N,91861374-1fe7-4e16-b696-48b868c9b93d
983,-80.480202,43.448642,8968,1423,2009-12-15 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,6863,WEBER ST E,40880,150.0,NaN,NaN,N,195fa0dd-c095-46b4-b8e0-c6909bb9e979
2086,-80.573004,43.443999,10071,1035,2011-12-15 15:38:27+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,0,UNKNOWN,0,150.0,NaN,NaN,N,21fba635-a684-4fd3-8a9e-f75eee7349ca
1549,-80.495598,43.437140,9534,99,2000-01-07 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,6450,ST CLAIR AVE,35730,150.0,NaN,NaN,N,134284c5-690d-42cb-b3fc-6211d4910b57
1056,-80.479118,43.432772,9041,127,1998-12-18 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,10877,OTTAWA ST S,29680,150.0,NaN,NaN,N,ac0f49c2-3e7e-4ccd-8fbc-e539e4778ae3
753,-80.480505,43.430662,8738,267,2005-07-20 00:00:00+00:00,MAIN,8-12 hours,REPAIR COMPLETED,UNKNOWN,OTHER,...,Y,CATEGORY 1,11743,OTTAWA ST S,29660,150.0,NaN,NaN,N,346376f6-1b5e-41dc-9b2a-bbe387e68717


From reading different studies, it seems as though the year the pipe was installed or rather the age of the pipe is a critical factor in predicting breaks/time of failure. In this case, I believe I may have to drop the observations where no year is indicated. 

In [213]:
break_data.dropna(axis=0, subset=['ASSET_YEAR_INSTALLED'], inplace=True)

In [214]:
print(break_data.shape)
print_null_values(break_data)

(2585, 25)
LONGITUDE -  0
LATITUDE -  0
OBJECTID -  0
WATBREAKINCIDENTID -  0
INCIDENT_DATE -  0
BREAK_TYPE -  0
HOUR_IMPACTED -  0
STATUS -  0
BREAK_NATURE -  0
BREAK_APPARENT_CAUSE -  0
POSITIVE_PRESSURE_MAINTANED -  0
AIR_GAP_MAINTANED -  0
MECHANICAL_REMOVAL -  0
FLUSHING_EXCAVATION -  0
HIGHER_VELOCITY_FLUSHING -  0
ANODE_INSTALLED -  0
BREAK_CATEGORIZATION -  0
ROADSEGMENTID -  0
STREET -  0
ASSETID -  0
ASSET_SIZE -  0
ASSET_YEAR_INSTALLED -  0
ASSET_MATERIAL -  0
ASSET_EXISTS -  0
GLOBALID -  0


Lucky for us that took care of our missing values in the `ASSET_MATERIAL` column. Now we have a clean dataset finally to do some EDA and feature engineering.

In [215]:
break_data.to_csv('../data/processed/cleaned_break_data.csv', index=False)